# 檢查輸出

In [1]:
import os
import openai
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

openai.api_key  = os.environ['OPENAI_API_KEY']

In [5]:
def get_completion_from_messages(messages, model="gpt-3.5-turbo", temperature=0, max_tokens=2700):
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=temperature, 
        max_tokens=max_tokens, 
    )
    return response.choices[0].message["content"]

### 檢查可能的有害內容

In [6]:
final_response_to_customer = f"""
以上是我們店裡的 SmartX Pro 手機和 FotoSnap 單眼相機的資訊。SmartX Pro 手機擁有6.1吋顯示器、128GB 儲存空間、12MP 雙鏡頭和5G 網路等功能。FotoSnap 單眼相機則擁有24.2MP 感光元件、1080p 影片、3 吋 LCD 和可更換鏡頭等功能。

此外，我們店裡還有多款電視產品可供選擇。CineView 4K 液晶電視擁有55吋顯示器、4K 解析度、HDR 和智慧電視等功能。CineView 8K 液晶電視則擁有65吋顯示器、8K 解析度、HDR 和智慧電視等功能。CineView OLED 電視則擁有55吋顯示器、4K 解析度、HDR 和智慧電視等功能。此外，我們還有 SoundMax 家庭劇院和 SoundMax 音箱等產品，可以升級您的電視聲光效果。

如果您對這些產品有任何問題或需要更多資訊，請隨時向我們詢問。
"""
response = openai.Moderation.create(
    input=final_response_to_customer
)
moderation_output = response["results"][0]
print(moderation_output)

{
  "categories": {
    "harassment": false,
    "harassment/threatening": false,
    "hate": false,
    "hate/threatening": false,
    "self-harm": false,
    "self-harm/instructions": false,
    "self-harm/intent": false,
    "sexual": false,
    "sexual/minors": false,
    "violence": false,
    "violence/graphic": false
  },
  "category_scores": {
    "harassment": 8.845762e-09,
    "harassment/threatening": 2.1843086e-08,
    "hate": 1.0149355e-09,
    "hate/threatening": 6.4688717e-09,
    "self-harm": 2.2744202e-09,
    "self-harm/instructions": 4.1273363e-09,
    "self-harm/intent": 1.8380901e-10,
    "sexual": 1.2166727e-06,
    "sexual/minors": 1.5611114e-08,
    "violence": 1.6215482e-07,
    "violence/graphic": 1.8128114e-08
  },
  "flagged": false
}


### 檢查輸出的訊息是否有如實按照產品資訊產生

In [9]:
system_message = f"""
您是一位助理，主要負責評估客戶服務代表的回答是否足夠滿足客戶的問題， \
並且也驗證所有助理引用的產品資訊是否正確。 \
產品資訊以及用戶與客戶服務代表的訊息將以3個反引號（即```）進行區隔。 \
只需用Y或N字母來回應，不需要任何標點符號： \
如果輸出充分地回答了問題，並且回應正確地使用了產品資訊，則回答「Y」； \
否則，則回答「N」。

只輸出單一字母。
"""

customer_message = f"""
告訴我有關SmartX ProPhone和FotoSnap相機的資訊，
就是那款數位單眼相機的。另外也告訴我你們的電視有哪些。
"""

product_information = """
{
    "SmartX ProPhone": {
        "name": "SmartX ProPhone",
        "category": "手機和配件",
        "brand": "SmartX",
        "model_number": "SX-PP10",
        "warranty": "一年",
        "rating": 4.6,
        "features": ["6.1 吋顯示器", "128GB 儲存空間", "12MP 雙鏡頭", "5G 網路"],
        "description": "一款功能強大並具有先進照相功能的智慧手機",
        "price": 899.99
    },
    "MobiTech 行動盔甲": {
        "name": "MobiTech 行動電源",
        "category": "手機和配件",
        "brand": "MobiTech",
        "model_number": "MT-PC20",
        "warranty": "一年",
        "rating": 4.3,
        "features": ["5000mAh 電池", "無線充電", "與SmartX ProPhone相容"],
        "description": "一款具有內置電池以及堅固外殼可延長您使用時間的行動電源。",
        "price": 59.99
    },
    "SmartX MiniPhone": {
        "name": "SmartX MiniPhone",
        "category": "手機和配件",
        "brand": "SmartX",
        "model_number": "SX-MP5",
        "warranty": "一年",
        "rating": 4.2,
        "features": ["4.7 吋顯示器", "64GB 儲存空間", "8MP 鏡頭", "4G 網路"],
        "description": "一款小巧且實惠的智慧手機，適合基本任務使用。",
        "price": 399.99
    },
    "MobiTech 無線充電器": {
        "name": "MobiTech 無線充電器",
        "category": "手機和配件",
        "brand": "MobiTech",
        "model_number": "MT-WC10",
        "warranty": "一年",
        "rating": 4.5,
        "features": ["10W 快速充電", "與Qi充電標準相容", "LED 指示燈", "精巧設計"],
        "description": "一款方便的無線充電器，讓您的工作區域無雜亂的電線。",
        "price": 29.99
    },
    "SmartX 耳機": {
        "name": "SmartX 耳機",
        "category": "手機和配件",
        "brand": "SmartX",
        "model_number": "SX-EB20",
        "warranty": "一年",
        "rating": 4.4,
        "features": ["真無線", "藍牙  5.0", "觸控控制", "24 小時播放時間"],
        "description": "一款輕便且音質出色的無線耳機，讓您隨時隨地都能享受音樂。",
        "price": 99.99
    },

    "CineView 4K 液晶電視": {
        "name": "CineView 4K 液晶電視",
        "category": "電視和家庭劇院系統",
        "brand": "CineView",
        "model_number": "CV-4K55",
        "warranty": "兩年",
        "rating": 4.8,
        "features": ["55 吋顯示器", "4K 解析度", "HDR", "智慧電視"],
        "description": "一部色彩鮮豔，並擁有智能功能的令人驚嘆的 4K 電視。",
        "price": 599.99
    },
    "SoundMax 家庭劇院": {
        "name": "SoundMax 家庭劇院",
        "category": "電視和家庭劇院系統",
        "brand": "SoundMax",
        "model_number": "SM-HT100",
        "warranty": "一年",
        "rating": 4.4,
        "features": ["5.1 聲道", "1000W 輸出", "無線重低音", "藍牙"],
        "description": "一部強大的家庭劇院系統，為您提供身臨其境的音效體驗。",
        "price": 399.99
    },
    "CineView 8K 液晶電視": {
        "name": "CineView 8K 液晶電視",
        "category": "電視和家庭劇院系統",
        "brand": "CineView",
        "model_number": "CV-8K65",
        "warranty": "兩年",
        "rating": 4.9,
        "features": ["65 吋顯示器", "8K 解析度", "HDR", "智慧電視"],
        "description": "用這部令人驚嘆的 8K 電視體驗電視工藝的未來。",
        "price": 2999.99
    },
    "SoundMax 音箱": {
        "name": "SoundMax 音箱",
        "category": "電視和家庭劇院系統",
        "brand": "SoundMax",
        "model_number": "SM-SB50",
        "warranty": "一年",
        "rating": 4.3,
        "features": ["2.1 聲道", "300W 輸出", "無線重低音", "藍牙"],
        "description": "使用這款時尚且音質出色的音響升級您的電視聲光效果。",
        "price": 199.99
    },
    "CineView OLED 電視": {
        "name": "CineView OLED 電視",
        "category": "電視和家庭劇院系統",
        "brand": "CineView",
        "model_number": "CV-OLED55",
        "warranty": "兩年",
        "rating": 4.7,
        "features": ["55 吋顯示器", "4K 解析度", "HDR", "智慧電視"],
        "description": "這款 OLED 電視讓您體驗什麼是真正的黑色以及色彩。",
        "price": 1499.99
    },

    "FotoSnap 單眼相機": {
        "name": "FotoSnap 單眼相機",
        "category": "相機和攝影機",
        "brand": "FotoSnap",
        "model_number": "FS-DSLR200",
        "warranty": "一年",
        "rating": 4.7,
        "features": ["24.2MP 感光元件", "1080p 影片", "3 吋 LCD", "可更換鏡頭"],
        "description": "使用這款多功能的單眼相機捕捉精彩照片和視頻。",
        "price": 599.99
    },
    "ActionCam 4K": {
        "name": "ActionCam 4K",
        "category": "相機和攝影機",
        "brand": "ActionCam",
        "model_number": "AC-4K",
        "warranty": "一年",
        "rating": 4.4,
        "features": ["4K 影片", "防水", "影像穩定技術", "Wi-Fi"],
        "description": "使用這款堅固且精巧的 4K 動作攝影機記錄您的冒險。",
        "price": 299.99
    },
    "FotoSnap 無反光鏡相機": {
        "name": "FotoSnap 無反光鏡相機",
        "category": "相機和攝影機",
        "brand": "FotoSnap",
        "model_number": "FS-ML100",
        "warranty": "一年",
        "rating": 4.6,
        "features": ["20.1MP 感光元件", "4K 影片", "3 吋觸控螢幕", "可更換鏡頭"],
        "description": "這款具有先進功能的精巧輕量級無反光鏡相機。",
        "price": 799.99
    },
    "ZoomMaster 攝影機": {
        "name": "ZoomMaster 攝影機",
        "category": "相機和攝影機",
        "brand": "ZoomMaster",
        "model_number": "ZM-CM50",
        "warranty": "一年",
        "rating": 4.3,
        "features": ["1080p 影片", "30x 光學變焦", "3 吋LCD", "影像穩定技術"],
        "description": "用這款易於使用的攝影機捕捉生活的時刻。",
        "price": 249.99
    },
    "FotoSnap 拍立得相機": {
        "name": "FotoSnap 拍立得相機",
        "category": "相機和攝影機",
        "brand": "FotoSnap",
        "model_number": "FS-IC10",
        "warranty": "一年",
        "rating": 4.1,
        "features": ["立即列印", "內建閃光燈", "自拍鏡", "可攜式電池"],
        "description": "用這款有趣和便攜的拍立得相機創造即時回憶。",
        "price": 69.99
    }
}
"""
q_a_pair = f"""
客戶訊息：{customer_message}
產品資訊：{product_information}
客服回覆：{final_response_to_customer}

回覆是否正確地使用了查詢到的資訊？
回覆是否足夠地回答了問題？

輸出 Y 或 N
"""
messages = [
    {'role': 'system', 'content': system_message},
    {'role': 'user', 'content': q_a_pair}
]

response = get_completion_from_messages(messages, max_tokens=1)
print(response)

Y
